In [128]:
import pandas as pd 
import psycopg2
import altair as alt
import datetime as dt 

In [102]:
db_host = "energy-monitor.c57vkec7dkkx.eu-west-2.rds.amazonaws.com"
db_username = "postgres"
db_password = "password"
db_port = "5432"
db_name = "postgres" 

In [116]:
conn = psycopg2.connect(
           host=db_host,
           database=db_name,
           user=db_username,
           password=db_password,
           port=db_port
       )


Demand_query = "SELECT * FROM Demand" 
Gen_query = "SELECT * FROM Generation" 
Cost_query = "SELECT * FROM Cost"
carbon_query = "SELECT * FROM Carbon"
Demand_df = pd.read_sql_query(Demand_query, conn)
Cost_df = pd.read_sql_query(Cost_query, conn)
Gen_df = pd.read_sql_query(Gen_query, conn)
carbon_df = pd.read_sql_query(carbon_query, conn)


if conn:
    conn.close()
    print("Database connection closed.")

/var/folders/81/f1rdgkd103zcv48_n3ps90c40000gp/T/ipykernel_61773/2580455365.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Demand_df = pd.read_sql_query(Demand_query, conn)
/var/folders/81/f1rdgkd103zcv48_n3ps90c40000gp/T/ipykernel_61773/2580455365.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Cost_df = pd.read_sql_query(Cost_query, conn)
/var/folders/81/f1rdgkd103zcv48_n3ps90c40000gp/T/ipykernel_61773/2580455365.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Gen_df = pd.read_sql_query(Gen_query, conn)


Database connection closed.


/var/folders/81/f1rdgkd103zcv48_n3ps90c40000gp/T/ipykernel_61773/2580455365.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  carbon_df = pd.read_sql_query(carbon_query, conn)


In [104]:
#print(Demand_df.head())
#rint(Cost_df.head())
#print(Gen_df)
#print(carbon_df.loc[carbon_df["carbon_level"]=="moderate"])
#print(carbon_df.loc[carbon_df["carbon_level"] == "low"])
#groups = Gen_df["fuel_type"].unique()
#print(groups)
    

In [117]:
chart = alt.Chart(Gen_df[["publish_time", "generated", "fuel_type"]]).mark_line().encode(
    x="publish_time", y="generated",color="fuel_type")
chart


alt.Chart(...)

In [118]:
fuel_type_mapping = {
    "BIOMASS": "BIOMASS",
    "CCGT": "CCGT (Combined Cycle Gas Turbine)",
    "COAL": "COAL",
    "NPSHYD": "NPSHYD (Non-Pumped Storage Hydro)",
    "NUCLEAR": "NUCLEAR",
    "OCGT": "OCGT (Open Cycle Gas Turbine)",
    "OIL": "OIL",
    "OTHER": "OTHER",
    "PS": "PS (Pumped Storage Hydro)",
    "WIND": "WIND"
}

interconnector_mapping = {
    "INTELEC": "England(INTELEC)",
    "INTEW": "Wales(INTEW)",
    "INTFR": "France(INTFR)",
    "INTGRNL": "Greenland(INTGRNL)",
    "INTIFA2": "Ireland-Scotland(INTIFA2)",
    "INTIRL": "Ireland(INTIRL)",
    "INTNED": "Netherlands(INTNED)",
    "INTNEM": "Belgium(INTNEM)",
    "INTNSL": "Norway(INTNSL)",
    "INTVKL": "Denmark(INTVKL)"
}

In [107]:
fuel_types = ["BIOMASS", "CCGT", "COAL", "NPSHYD",
              "NUCLEAR", "OCGT", "OIL", "OTHER", "PS", "WIND"]
interconnectors = ["INTELEC", "INTEW", "INTFR", "INTGRNL",
                   "INTIFA2", "INTIRL", "INTNED", "INTNEM", "INTNSL", "INTVKL"]
df_fuel_types = Gen_df[Gen_df['fuel_type'].isin(fuel_type_mapping.keys())].copy()
df_fuel_types['fuel_type'] = df_fuel_types['fuel_type'].map(fuel_type_mapping)


df_interconnectors = Gen_df[Gen_df['fuel_type'].isin(interconnector_mapping)].copy()
df_interconnectors['fuel_type'] = df_interconnectors['fuel_type'].map(interconnector_mapping)
df_interconnectors.rename(columns={'fuel_type': 'country'}, inplace=True)

In [119]:
chart = alt.Chart(df_fuel_types[["publish_time", "generated", "fuel_type"]]).mark_line().encode(
    x="publish_time", y="generated", color="fuel_type")
chart

alt.Chart(...)

In [120]:
chart = alt.Chart(df_interconnectors[["publish_time", "generated", "country"]]).mark_line().encode(
    x="publish_time", y="generated", color="country")
chart

alt.Chart(...)

In [110]:
carbon_chart = alt.Chart(carbon_df).mark_line().encode(alt.X('publish_time:T', title='Time'),
            y=alt.Y('forecast:Q', title='Carbon Forecast'))

carbon_chart

alt.Chart(...)

In [111]:
demand_chart = alt.Chart(Demand_df).mark_line().encode(alt.X('publish_time:T', title='Time'),
                                                       y=alt.Y('demand_amt:Q', title='Demand'))
demand_chart

alt.Chart(...)

In [121]:
carbon_chart_bar = alt.Chart(carbon_df).mark_bar().encode(alt.X('publish_time:T', title='Time'),
                                                       y=alt.Y('forecast:Q', title='Carbon Forecast'),color='carbon_level')
carbon_chart_bar

alt.Chart(...)

In [145]:
print(carbon_df.loc[carbon_df["forecast"]==min(carbon_df["forecast"].values)])
print(carbon_df.loc[carbon_df["forecast"] == max(
    carbon_df["forecast"].values)])

          publish_time  forecast carbon_level
30 2024-08-16 14:00:00        58          low
          publish_time  forecast carbon_level
15 2024-08-16 06:30:00       115     moderate


In [137]:
max_publish_time = carbon_df.loc[carbon_df["forecast"] == max(
    carbon_df["forecast"].values), "publish_time"].dt.time.iloc[0]


06:30:00


In [147]:
min_publish_time = carbon_df.loc[carbon_df["forecast"] == min(
    carbon_df["forecast"].values), "publish_time"].dt.time.iloc[0]
min_publish_time
min(carbon_df["forecast"].values)

TypeError: 'numpy.ndarray' object is not callable

In [138]:

[{'Biomass': 120, 'Coal': 937, 'Dutch Imports': 474, 'French Imports': 53,
    'Gas (Combined Cycle)': 394, 'Gas (Open Cycle)': 651, 'Hydro': 0, 'Irish Imports': 458, 'Nuclear': 0, 'Oil': 935, 'Other': 300, 'Pumped Storage': 0, 'Solar': 0, 'Wind': 0}]

{'data': [{'Biomass': 120, 'Coal': 937, 'Dutch Imports': 474, 'French Imports': 53, 'Gas (Combined Cycle)': 394, 'Gas (Open Cycle)': 651, 'Hydro': 0, 'Irish Imports': 458, 'Nuclear': 0, 'Oil': 935, 'Other': 300, 'Pumped Storage': 0, 'Solar': 0, 'Wind': 0}]}


In [148]:
data = pd.DataFrame({'Biomass': 120, 'Coal': 937, 'Dutch Imports': 474, 'French Imports': 53,
    'Gas (Combined Cycle)': 394, 'Gas (Open Cycle)': 651, 'Hydro': 0, 'Irish Imports': 458, 'Nuclear': 0, 'Oil': 935, 'Other': 300, 'Pumped Storage': 0, 'Solar': 0, 'Wind': 0})
data

ValueError: If using all scalar values, you must pass an index

In [150]:
intensity_factors = {'Energy Source': ['Biomass', 'Coal', 'Dutch Imports', 'French Imports', 'Gas (Combined Cycle)', 'Gas (Open Cycle)', 
                      'Hydro', 'Irish Imports', 'Nuclear', 'Oil', 'Other', 'Pumped Storage', 'Solar', 'Wind'],
    'Intensity (gCO₂/kWh)': [120, 937, 474, 53, 394, 651, 0, 458, 0, 935, 300, 0, 0, 0],
    'Explanation': [
        'Uses organic materials which result in moderate emissions.',
        'High carbon emissions due to combustion of fossil fuels.',
        'Mixed source imports with moderate carbon impact.',
        'Imported nuclear and renewable energy with low emissions.',
        'More efficient gas-burning technology, lower emissions.',
        'Less efficient gas technology, higher emissions.',
        'Zero emissions as it relies on water flow.',
        'Mixed source imports with moderate carbon impact.',
        'Zero emissions from nuclear reactions.',
        'High emissions from oil combustion.',
        'Varied sources with different emissions levels.',
        'Storage method that does not emit carbon.',
        'Zero emissions harnessing solar power.',
        'Zero emissions capturing wind energy.']}
df = pd.DataFrame(intensity_factors)
df


,Energy Source,Intensity (gCO₂/kWh),Explanation
0,Biomass,120,Uses organic materials which result in moderat...
1,Coal,937,High carbon emissions due to combustion of fos...
2,Dutch Imports,474,Mixed source imports with moderate carbon impact.
3,French Imports,53,Imported nuclear and renewable energy with low...
4,Gas (Combined Cycle),394,"More efficient gas-burning technology, lower e..."
5,Gas (Open Cycle),651,"Less efficient gas technology, higher emissions."
6,Hydro,0,Zero emissions as it relies on water flow.
7,Irish Imports,458,Mixed source imports with moderate carbon impact.
8,Nuclear,0,Zero emissions from nuclear reactions.
9,Oil,935,High emissions from oil combustion.


In [ ]:

"The demand and generation by fuel type (gas, coal, wind, nuclear, solar etc.) for each region is forecast several days ahead at 30-min temporal resolution using an ensemble of state-of-the-art supervised Machine Learning (ML) regression models. An advanced model ensembling technique is used to blend the ML models to generate a new optimised meta-model. The forecasts are updated every 30 mins using a nowcasting technique to adjust the forecasts a short period ahead.
The carbon intensity factors are applied to each technology type for each import generation mix to
calculate the forecast"